In [1]:
# Install necessary libraries
!pip install pandas nltk
!pip install ipywidgets
!pip install spotipy
!pip install langdetect

# Importing libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import numpy as np
import string
from IPython.display import display, Markdown
import ipywidgets as widgets

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("Setup complete. Ready to load data.")

Setup complete. Ready to load data.


[nltk_data] Downloading package punkt to /Users/anita/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/anita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/anita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/anita/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


##### Clean Data

In [2]:
# Import the Spotipy library
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initialize Spotipy with Spotify Developer Credentials
client_id = '1075d2afec3d414ca02ec7002f1d5576'  # Replace with your actual Spotify Client ID
client_secret = 'bb1fd201730b484983d952ecbd344b33'  # Replace with your actual Spotify Client Secret

credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=credentials)

In [3]:
# Load data from a CSV file
data = pd.read_csv('/Users/anita/Documents/NLPSongs-/music_data.csv')
print("Data loaded successfully!")
print("Sample data:")
data # Display the first few rows of the dataset to verify it's loaded correctly

Data loaded successfully!
Sample data:


,artist,title,id,release_date,lyrics
0,Ariana Grande,"​thank u, next",4063065,2018-11-03,[Verse 1]\nThought I'd end up with Sean\nBut h...
1,Ariana Grande,7 rings,4067762,2019-01-18,"[Verse 1]\nYeah, breakfast at Tiffany's and bo..."
2,Ariana Grande,​God is a woman,3681280,2018-07-13,"[Chorus]\nYou, you love it how I move you\nYou..."
3,Ariana Grande,Side To Side,2457495,2016-05-20,[Intro: Ariana Grande & Nicki Minaj]\nI've bee...
4,Ariana Grande,​​no tears left to cry,3649172,2018-04-20,"[Intro]\nRight now, I'm in a state of mind\nI ..."
...,...,...,...,...,...
3739,Taylor Swift,"Taylor Swift - no body, no crime (Traducción a...",6274728,2020-12-11,[Intro: HAIM]\nÉl lo hizo\nÉl lo hizo\n\n[Vers...
3740,Taylor Swift,Welcome Back Grunwald,6226864,NaN,Turn WYCD on\nYou're\nOn your Grunwald\nBack f...
3741,Taylor Swift,Tolerate it (Polskie Tłumaczenie),6315848,2020-12-11,[Zwrotka 1]\nSiedzę i patrzę jak czytasz z gło...
3742,Taylor Swift,Find you,6209316,NaN,Trying just like they say\nJust taking the ste...


In [4]:
null_values = data.isnull().sum()
print(null_values)

artist            0
title             0
id                0
release_date    993
lyrics            0
dtype: int64


In [5]:
#Remove null values
data = data.dropna()

In [6]:
duplicate_rows = data.duplicated().sum()
print(duplicate_rows)

0


In [7]:
from langdetect import detect

def is_english(row):
    for column in row:
        try:
            if detect(str(column)) != 'en':
                return False
        except:
            pass
    return True

mask = data.apply(is_english, axis=1)
data = data[mask]

#### Preprocess

##### Tokenization and Lemmatization 

In [20]:
# Function to preprocess data
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    return ' '.join(tokens)

# Apply preprocessing to the lyrics column
data['processed_lyrics'] = data['lyrics'].apply(preprocess_text)
print("Data preprocessing complete. Here's a preview:")
print(data[['lyrics', 'processed_lyrics']].head())

Data preprocessing complete. Here's a preview:
                                                lyrics  \
517  [Chorus: Charlie Puth]\nWe don't talk anymore,...   
518  [Intro]\nAlright\nOoh, yeah\n\n[Verse 1]\nI'll...   
519  [Intro: Charlie Puth]\nLet's Marvin Gaye and g...   
520  [Chorus]\nI'm only one call away\nI'll be ther...   
522  [Verse 1]\nYeah, maybe I'ma get a little anxio...   

                                      processed_lyrics  
517  [ chorus : charlie puth ] we do n't talk anymo...  
518  [ intro ] alright ooh , yeah [ verse 1 ] i 'll...  
519  [ intro : charlie puth ] let 's marvin gaye an...  
520  [ chorus ] i 'm only one call away i 'll be th...  
522  [ verse 1 ] yeah , maybe i'ma get a little anx...  


In [21]:
data['processed_lyrics'].head(50)

517    [ chorus : charlie puth ] we do n't talk anymo...
518    [ intro ] alright ooh , yeah [ verse 1 ] i 'll...
519    [ intro : charlie puth ] let 's marvin gaye an...
520    [ chorus ] i 'm only one call away i 'll be th...
522    [ verse 1 ] yeah , maybe i'ma get a little anx...
523    [ verse 1 : charlie puth ] no , this is not go...
526    [ verse 1 ] it started with a kiss on your mam...
528    [ chorus ] i warned myself that i should n't p...
531    [ verse 1 ] he 's such a nice boy , so well-ma...
533    [ chorus ] do n't run from me , river do n't r...
535    [ verse 1 ] i got a situation , this girl been...
536    [ verse 1 ] i was just with you on your birthd...
538    [ verse 1 ] oh , the way that you dance on me ...
539    [ verse 1 ] you could say i lived a crazy life...
541    [ verse 1 ] when i 'm old and grown i wo n't s...
545    [ intro ] mmm , mmm , mmm [ verse 1 ] somebody...
548    [ verse 1 ] well , i get a thrill from down my...
549    [ intro ] ohh , ooh [ ve

In [22]:
# Remove rows with empty lyrics or that contain only one word
data = data[data['processed_lyrics'].str.split().str.len() > 1]

#### Spotify Connection

In [23]:
def search_spotify(song_title, artist):
    query = f"track:{song_title} artist:{artist}"
    results = spotify.search(q=query, type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'preview_url': track['preview_url'],
            'spotify_url': track['external_urls']['spotify']
        }
    else:
        return "No results found on Spotify."

In [24]:
def retrieve_song_and_search_spotify(lyrics_input):
    processed_input = preprocess_text(lyrics_input)
    match = data[data['processed_lyrics'].str.contains(processed_input, na=False)]
    if not match.empty:
        song_title = match.iloc[0]['title']
        artist = match.iloc[0]['artist']
        spotify_info = search_spotify(song_title, artist)
        display_spotify_result(spotify_info)  # Using the new display function
    else:
        display(Markdown("**No matching song found.**"))

In [25]:
from IPython.display import display, HTML, Audio

def display_spotify_result(spotify_info):
    if isinstance(spotify_info, dict):
        html_content = f"<h3>{spotify_info['name']} by {spotify_info['artist']}</h3>"
        html_content += f"<p><strong>Album:</strong> {spotify_info['album']}</p>"
        html_content += f"<a href='{spotify_info['spotify_url']}' target='_blank'>Listen on Spotify</a>"
        if spotify_info['preview_url']:
            display(HTML(html_content))
            display(Audio(spotify_info['preview_url'], autoplay=False))
        else:
            display(HTML(html_content))
    else:
        print(spotify_info)

#### Search

In [26]:
import ipywidgets as widgets
from IPython.display import display

# Input for lyrics
lyrics_input = widgets.Text(
    value='',
    placeholder='Type lyrics here...',
    description='Lyrics:',
    disabled=False
)

# Button to trigger the search
search_button = widgets.Button(
    description='Search',
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    icon='search'  # FontAwesome icon name (without 'fa-')
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        retrieve_song_and_search_spotify(lyrics_input.value)

search_button.on_click(on_button_clicked)

display(lyrics_input, search_button, output)

Text(value='', description='Lyrics:', placeholder='Type lyrics here...')

Button(button_style='info', description='Search', icon='search', style=ButtonStyle())

Output()

we need more preprocessing, show metrics of the model, try and remove duplicates from the dataset in order to get more nuanced results, look to improve performance in every way possible, refine the data so that it is all fully in English